In [3]:
from transformers import AutoTokenizer, AutoModelWithHeads

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelWithHeads.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
import json
with open('/home/vmanuel/typing_network/data/3_types/train.json', 'r') as inp:
    lines = [json.loads(l) for l in inp.readlines()]

In [5]:
sentences = [' '.join(l['left_context_token']) + ' [SEP] ' + l['mention_span'] + ' [SEP] ' + ' '.join(l['right_context_token']) for l in lines]

In [9]:
tokenized_sent = [tokenizer(s, max_length=80, truncation=True, padding="max_length")['input_ids'] for s in sentences]

In [39]:
tokenizer(sentences[0], max_length=80, truncation=True, padding="max_length")

{'input_ids': [101, 1996, 11203, 8480, 1999, 7564, 2044, 2197, 2095, 1005, 1055, 14734, 1011, 25537, 3947, 1024, 102, 1996, 102, 2231, 10035, 9781, 6434, 2012, 1021, 1012, 3429, 4551, 5747, 9050, 1010, 2039, 4868, 1003, 2013, 2197, 2991, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [11]:
from torch.utils.data import Dataset, DataLoader

In [18]:
class bertDataset(Dataset):
    def __init__(self, sent):
        self.sent = sent
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.tokenize()
    def tokenize(self):
        self.tokenized_sent = [tokenizer(s, max_length=80, truncation=True, padding="max_length")['input_ids'] for s in sentences]
    def __getitem__(self, idx):
        return self.tokenized_sent[idx]
    def __len__(self):
        return len(self.tokenized_sent)

In [19]:
bd = bertDataset(sentences)

In [20]:
datal = DataLoader(bd, batch_size = 10)

In [35]:
import torch

for s in datal:
    print(len(s))
    s = torch.stack(s)
    o = model(s)
    break

80


In [37]:
o

(tensor([[[-0.7147,  0.3159,  0.4077,  ...,  0.2102,  0.4964,  0.1172],
          [-0.6580,  0.3274,  0.4390,  ...,  0.2261,  0.5316,  0.1162],
          [-0.7035,  0.3250,  0.4171,  ...,  0.2144,  0.5066,  0.1140],
          ...,
          [-0.7086,  0.3135,  0.4019,  ...,  0.2122,  0.5002,  0.1068],
          [-0.7126,  0.3140,  0.3972,  ...,  0.2108,  0.4998,  0.1107],
          [-0.7137,  0.3115,  0.4024,  ...,  0.2122,  0.5027,  0.1088]],
 
         [[-0.1052,  0.0305,  0.0765,  ..., -0.1037,  0.1553,  0.2072],
          [ 0.8634,  0.2332, -0.3373,  ...,  0.2747, -0.7856, -0.2748],
          [ 0.0284,  0.1590, -0.2177,  ..., -0.0360,  0.3692,  0.1448],
          ...,
          [-0.2214, -0.1209,  0.2956,  ..., -0.2651,  0.5146, -0.3838],
          [-0.5641, -0.5534, -0.1680,  ...,  0.4389,  0.5160, -0.0312],
          [ 0.8322,  0.2377, -0.3158,  ...,  0.2851, -0.7825, -0.2799]],
 
         [[-0.0478,  0.1768,  0.0335,  ..., -0.1416,  0.1776,  0.0878],
          [-0.1153, -0.0252,

In [34]:
a.shape

torch.Size([80, 768])

In [27]:
with open('/datahdd/vmanuel/entity_typing_all_datasets/data/ontology/types.txt', 'r') as inp:
    types = [t.replace('\n', '') for t in inp.readlines()]

In [28]:
len(types)

10331

In [29]:
types

['person',
 'group',
 'organization',
 'location',
 'entity',
 'time',
 'object',
 'event',
 'place',
 'accident',
 'actor',
 'agency',
 'airline',
 'airplane',
 'airport',
 'animal',
 'architect',
 'army',
 'art',
 'artist',
 'athlete',
 'attack',
 'author',
 'award',
 'biology',
 'body_part',
 'bridge',
 'broadcast',
 'broadcast_station',
 'building',
 'car',
 'cemetery',
 'chemistry',
 'city',
 'coach',
 'company',
 'computer',
 'conflict',
 'country',
 'county',
 'currency',
 'degree',
 'department',
 'director',
 'disease',
 'doctor',
 'drug',
 'education',
 'election',
 'engineer',
 'ethnic_group',
 'facility',
 'film',
 'finance',
 'food',
 'game',
 'geography',
 'god',
 'government',
 'health',
 'heritage',
 'holiday',
 'hospital',
 'hotel',
 'institution',
 'instrument',
 'internet',
 'island',
 'language',
 'law',
 'lawyer',
 'league',
 'legal',
 'leisure',
 'library',
 'living_thing',
 'mass_transit',
 'medicine',
 'military',
 'mobile_phone',
 'monarch',
 'mountain',
 'musi

In [4]:
import json
with open('/datahdd/vmanuel/entity_typing_all_datasets/data/entity_typing_original_datasets/choi/dev.json', 'r') as inp:
    exs = [json.loads(t) for t in inp.readlines()]

In [6]:
types = [e['y_str'] for e in exs]

In [10]:
all_types_dev = set([s for typ in types for s in typ])

In [15]:
len(all_types_dev)

5761

In [12]:
with open('/datahdd/vmanuel/entity_typing_all_datasets/data/entity_typing_original_datasets/choi/test_tree.json', 'r') as inp:
    exs = [json.loads(t) for t in inp.readlines()]
    types = [e['y_str'] for e in exs]
    all_types_test = set([s for typ in types for s in typ])

In [13]:
len(all_types_test)

1682

In [16]:
all_types = all_types_test.union(all_types_dev)

In [17]:
len(all_types)

5829

In [30]:
set(types).intersection(all_types)

{'recreation',
 'stock_car',
 'fatigue',
 'assistance',
 'rift',
 'theory',
 'cutting',
 'shoot',
 'experiment',
 'origin',
 'employee',
 'behavior',
 'body_hair',
 'domain_theory',
 'hostage',
 'memory_card',
 'illustration',
 'interview',
 'rifle',
 'source_language',
 'twin',
 'earl',
 'trail',
 'stranger',
 'today',
 'camera_movement',
 'count',
 'building_code',
 'page',
 'group_theory',
 'public',
 'worry',
 'spiritual',
 'tuition',
 'manuscript',
 'lady',
 'farewell',
 'shooting',
 'toy',
 'time_frame',
 'bureaucracy',
 'film_star',
 'order',
 'romance',
 'intensity',
 'circuit_board',
 'horror_story',
 'stop_motion',
 'chain_reaction',
 'rehabilitation',
 'metric',
 'excuse',
 'fly',
 'collection',
 'marriage',
 'gross',
 'space_race',
 'producer',
 'armor',
 'primary',
 'art_history',
 'joke',
 'banner_ad',
 'staple',
 'pride',
 'faithful',
 'breaking',
 'resistance',
 'toilet',
 'strategy',
 'market_share',
 'board',
 'exterior',
 'convenience',
 'regular',
 'balance',
 'brai

In [32]:
with open('/datahdd/vmanuel/entity_typing_all_datasets/data/entity_typing_original_datasets/choi/types_in_dev_and_test.txt', 'w') as out:
    for t in set(types).intersection(all_types):
        out.write(t)
        out.write('\n')